## Analysis for WomenTechWomenYes (WTWY) annual gala


- The goal of this analysis is to give the WomenTechWomenYes (WTWY) more information about the most appropriate subway stations in New York City by using MTA Turnstile data, so they can determine which days and locations for their street teams to collect most email addresses and people who sing up are sent free tickets to attend the WTWY summer gala event.


- The placement of street teams at entrances to subway stations will be determined according to MTA data analysis.

>**First,** I will do cleaning and importing data from http://web.mta.info/developers/turnstile.html


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math 
import random
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [150502,150509,150516,150523, 150530, 150606,150613, 150620, 150627, 150704, 150711,150718,150725]
turnstiles_df = get_data(week_nums)

In [3]:
turnstiles_df.to_csv('MTA_DATA.csv')

In [4]:
turnstiles_df.shape

(2496565, 11)

In [5]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,REGULAR,5106770,1729635
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,REGULAR,5106810,1729649
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,REGULAR,5106835,1729680
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,REGULAR,5106961,1729784
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,REGULAR,5107250,1729858


In [6]:
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
turnstiles_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [7]:
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

In [8]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,REGULAR,5106770,1729635,2015-04-25 00:00:00
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,REGULAR,5106810,1729649,2015-04-25 04:00:00
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,REGULAR,5106835,1729680,2015-04-25 08:00:00
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,REGULAR,5106961,1729784,2015-04-25 12:00:00
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,REGULAR,5107250,1729858,2015-04-25 16:00:00


In [9]:
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(20)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
506650,JFK03,R536,00-00-05,JFK JAMAICA CT1,2015-05-14 09:00:00,2
2016565,R258,R132,00-00-02,125 ST,2015-05-16 05:00:00,2
506615,JFK03,R536,00-00-05,JFK JAMAICA CT1,2015-05-08 21:00:00,2
1058151,N329,R201,00-03-04,WOODHAVEN BLVD,2015-05-20 05:00:00,2
2173748,R412,R146,00-03-03,HUNTS POINT AVE,2015-05-16 05:00:00,2
2169943,R412,R146,00-00-00,HUNTS POINT AVE,2015-05-16 05:00:00,2
506749,JFK03,R536,00-00-05,JFK JAMAICA CT1,2015-05-30 05:00:00,2
506748,JFK03,R536,00-00-05,JFK JAMAICA CT1,2015-05-30 01:00:00,2
2015471,R258,R132,00-00-00,125 ST,2015-05-16 05:00:00,2
339518,E003,R369,00-03-00,FT HAMILTON PKY,2015-06-01 13:00:00,2


In [10]:
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)


In [10]:
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(10)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,LEXINGTON AVE,2015-04-25 00:00:00,1
1664369,R147,R033,04-00-05,42 ST-TIMES SQ,2015-07-14 20:00:00,1
1664362,R147,R033,04-00-05,42 ST-TIMES SQ,2015-07-13 16:00:00,1
1664363,R147,R033,04-00-05,42 ST-TIMES SQ,2015-07-13 20:00:00,1
1664364,R147,R033,04-00-05,42 ST-TIMES SQ,2015-07-14 00:00:00,1
1664365,R147,R033,04-00-05,42 ST-TIMES SQ,2015-07-14 04:00:00,1
1664366,R147,R033,04-00-05,42 ST-TIMES SQ,2015-07-14 08:00:00,1
1664367,R147,R033,04-00-05,42 ST-TIMES SQ,2015-07-14 12:00:00,1
1664368,R147,R033,04-00-05,42 ST-TIMES SQ,2015-07-14 16:00:00,1
1664370,R147,R033,04-00-05,42 ST-TIMES SQ,2015-07-15 00:00:00,1


In [11]:
turnstiles_df.isna().sum()

C/A          0
UNIT         0
SCP          0
STATION      0
LINENAME     0
DIVISION     0
DATE         0
TIME         0
DESC         0
ENTRIES      0
EXITS        0
DATE_TIME    0
dtype: int64

In [12]:
turnstiles_df.shape

(2496549, 12)

In [13]:
turnstiles_daily = (turnstiles_df
                        .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False)
                        .ENTRIES.max())

In [14]:
turnstiles_daily

,C/A,UNIT,SCP,STATION,DATE,ENTRIES
0,A002,R051,02-00-00,LEXINGTON AVE,04/25/2015,5107620
1,A002,R051,02-00-00,LEXINGTON AVE,04/26/2015,5108367
2,A002,R051,02-00-00,LEXINGTON AVE,04/27/2015,5110015
3,A002,R051,02-00-00,LEXINGTON AVE,04/28/2015,5111702
4,A002,R051,02-00-00,LEXINGTON AVE,04/29/2015,5113477
...,...,...,...,...,...,...
413971,TRAM2,R469,00-05-01,RIT-ROOSEVELT,07/04/2015,5554
413972,TRAM2,R469,00-05-01,RIT-ROOSEVELT,07/05/2015,5554
413973,TRAM2,R469,00-05-01,RIT-ROOSEVELT,07/06/2015,5554
413974,TRAM2,R469,00-05-01,RIT-ROOSEVELT,07/07/2015,5554


In [15]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                                       .apply(lambda grp: grp.shift(1)))

<ipython-input-15-3a945374269e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily


In [15]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
0,A002,R051,02-00-00,LEXINGTON AVE,04/25/2015,5107620,NaN,NaN
1,A002,R051,02-00-00,LEXINGTON AVE,04/26/2015,5108367,04/25/2015,5107620.0
2,A002,R051,02-00-00,LEXINGTON AVE,04/27/2015,5110015,04/26/2015,5108367.0
3,A002,R051,02-00-00,LEXINGTON AVE,04/28/2015,5111702,04/27/2015,5110015.0
4,A002,R051,02-00-00,LEXINGTON AVE,04/29/2015,5113477,04/28/2015,5111702.0


In [16]:
turnstiles_daily.tail()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
413971,TRAM2,R469,00-05-01,RIT-ROOSEVELT,07/04/2015,5554,07/03/2015,5554.0
413972,TRAM2,R469,00-05-01,RIT-ROOSEVELT,07/05/2015,5554,07/04/2015,5554.0
413973,TRAM2,R469,00-05-01,RIT-ROOSEVELT,07/06/2015,5554,07/05/2015,5554.0
413974,TRAM2,R469,00-05-01,RIT-ROOSEVELT,07/07/2015,5554,07/06/2015,5554.0
413975,TRAM2,R469,00-05-01,RIT-ROOSEVELT,07/08/2015,5554,07/07/2015,5554.0


In [17]:
# Drop the rows for the earliest date in the df
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [18]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
1,A002,R051,02-00-00,LEXINGTON AVE,04/26/2015,5108367,04/25/2015,5107620.0
2,A002,R051,02-00-00,LEXINGTON AVE,04/27/2015,5110015,04/26/2015,5108367.0
3,A002,R051,02-00-00,LEXINGTON AVE,04/28/2015,5111702,04/27/2015,5110015.0
4,A002,R051,02-00-00,LEXINGTON AVE,04/29/2015,5113477,04/28/2015,5111702.0
5,A002,R051,02-00-00,LEXINGTON AVE,04/30/2015,5115219,04/29/2015,5113477.0


In [19]:
turnstiles_daily["STATION"][turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]]

3093       57 ST-7 AVE
3094       57 ST-7 AVE
3095       57 ST-7 AVE
3096       57 ST-7 AVE
3097       57 ST-7 AVE
              ...     
407889    FLATBUSH AVE
407890    FLATBUSH AVE
407891    FLATBUSH AVE
407892    FLATBUSH AVE
407893    FLATBUSH AVE
Name: STATION, Length: 2542, dtype: object

In [40]:
turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]].head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
3093,A011,R080,01-00-00,57 ST-7 AVE,04/26/2015,888989083,04/25/2015,888990856.0
3094,A011,R080,01-00-00,57 ST-7 AVE,04/27/2015,888987527,04/26/2015,888989083.0
3095,A011,R080,01-00-00,57 ST-7 AVE,04/28/2015,888984487,04/27/2015,888987527.0
3096,A011,R080,01-00-00,57 ST-7 AVE,04/29/2015,888981383,04/28/2015,888984487.0
3097,A011,R080,01-00-00,57 ST-7 AVE,04/30/2015,888978148,04/29/2015,888981383.0


In [28]:
turnstiles_daily['STATION'].value_counts()


34 ST-PENN STA     9088
FULTON ST          8679
42 ST-GRD CNTRL    5642
CANAL ST           5273
34 ST-HERALD SQ    4914
                   ... 
DYCKMAN-200 ST      270
NEREID AVE          264
MONTROSE AVE        258
CLEVELAND ST        182
PATH WTC 2           91
Name: STATION, Length: 384, dtype: int64